# Data Cleaning and Preparation

This notebook was created to clean the `Restaurant Health Inspection Results - NYC` for subsequent analysis in SQL and PowerBi.

The following approach was used to clean the data in order:

1. Initial inspection of the data

2. Standardising column names.

3. Check and remove duplicates

4. Type conversion. Convert date columns from strings to `datetime`.

5. Clean the string columns by removing empty spaces and converting `dba` from uppercase into title cases.

6. Standardising the inspection type. 
7. Cuisine standardisation into geographical origins and themes.

8. Categorised the violation code under their broader umbrellas.

9. Fill in missing values of `inspection_program`, `inspection_phase` and `grade`.

10. Drop irrelevant columns



## 1. Initial Data Inspection

In [1]:
# Import relevant libraries
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine


In [2]:
# Read the data
df = pd.read_csv("DOHMH_New_York_City_Restaurant_Inspection_Results_20250919.csv")
df.head(3)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50104876,NOODLE SUPER NO I,Manhattan,265,1 AVENUE,10003.0,2125290539,Chinese,08/24/2022,Violations were cited in the following area(s).,...,Pre-permit (Operational) / Second Compliance I...,40.732264,-73.981768,106.0,2.0,4800.0,1020423.0,1.009220e+09,MN21,NaN
1,50174387,TREATS OF KOREA,Queens,3150,STEINWAY ST,11103.0,8453232965,NaN,01/01/1900,NaN,...,NaN,40.760128,-73.918066,401.0,22.0,15500.0,4010534.0,4.006580e+09,QN70,NaN
2,50122756,MELLER'S SPORTS HUB & GRILL,Manhattan,1702,2 AVENUE,10128.0,9175964244,American,08/04/2025,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.779327,-73.950670,108.0,5.0,14602.0,1050066.0,1.015510e+09,MN32,NaN


In [3]:
# Check the shape of the dataframe
df.shape

(288888, 27)

In [4]:
# Check the dtypes
df.dtypes

CAMIS                      int64
DBA                       object
BORO                      object
BUILDING                  object
STREET                    object
ZIPCODE                  float64
PHONE                     object
CUISINE DESCRIPTION       object
INSPECTION DATE           object
ACTION                    object
VIOLATION CODE            object
VIOLATION DESCRIPTION     object
CRITICAL FLAG             object
SCORE                    float64
GRADE                     object
GRADE DATE                object
RECORD DATE               object
INSPECTION TYPE           object
Latitude                 float64
Longitude                float64
Community Board          float64
Council District         float64
Census Tract             float64
BIN                      float64
BBL                      float64
NTA                       object
Location Point1          float64
dtype: object

In [5]:
# Check the info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288888 entries, 0 to 288887
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  288888 non-null  int64  
 1   DBA                    288882 non-null  object 
 2   BORO                   288888 non-null  object 
 3   BUILDING               288379 non-null  object 
 4   STREET                 288885 non-null  object 
 5   ZIPCODE                285977 non-null  float64
 6   PHONE                  288882 non-null  object 
 7   CUISINE DESCRIPTION    285189 non-null  object 
 8   INSPECTION DATE        288888 non-null  object 
 9   ACTION                 285189 non-null  object 
 10  VIOLATION CODE         283037 non-null  object 
 11  VIOLATION DESCRIPTION  283037 non-null  object 
 12  CRITICAL FLAG          288888 non-null  object 
 13  SCORE                  272962 non-null  float64
 14  GRADE                  140844 non-nu

## 2. Columns clean up and standardisation

The columns have mixed cases. While some are in uppercases, others are in lower and title cases. They were cleaned and standardised to have the same format

In [6]:
# Check the column names
df.columns

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE', 'Latitude', 'Longitude',
       'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL',
       'NTA', 'Location Point1'],
      dtype='object')

In [7]:
# Standardise the column names
df.columns = (df.columns
                .str.strip()
                .str.lower()
                .str.replace(" ", "_")
              )
# check the effect
df.columns

Index(['camis', 'dba', 'boro', 'building', 'street', 'zipcode', 'phone',
       'cuisine_description', 'inspection_date', 'action', 'violation_code',
       'violation_description', 'critical_flag', 'score', 'grade',
       'grade_date', 'record_date', 'inspection_type', 'latitude', 'longitude',
       'community_board', 'council_district', 'census_tract', 'bin', 'bbl',
       'nta', 'location_point1'],
      dtype='object')

## 3. Checking and removing duplicates

I checked and removed duplicates based on `camis`, `inspection_date`, `violation_code`. Restauarants with the same identity, inspection date and violation codes were considered duplicates and removed.

In [8]:
# Check for duplicates based on camis, inspection_date, and violation_codes
df.duplicated(subset=["camis", "inspection_date", "violation_code"]).sum()


21

In [9]:
# Check and remove duplicates
# remove duplicates were camis, inspection_date, and violation_codes are the same
df.drop_duplicates(subset=["camis", "inspection_date", "violation_code"], inplace = True)
df.shape

(288867, 27)

21 duplicates were removed

## 4. Type Conversion

The date columns; `inspection_date`, `grade_date` and `record_date` were convereted to datetime types.

According to the data dictionary, inspection dates with values `01/01/1900` are actually null values. `01/01/1900` only served as a placeholder and so were appropriately converted to `NaN`.

In [10]:
# Convert the date columns to the datetime 
df["inspection_date"] = pd.to_datetime(df["inspection_date"])
df["grade_date"] = pd.to_datetime(df["grade_date"])
df["record_date"] = pd.to_datetime(df["record_date"])

# Replace empty inspection date ('01/01/1900') with NaN. From the data dictionary, '01/01/1900' represents missing dates
df["inspection_date"] = df["inspection_date"].replace("01/01/1900", np.NaN)

df.dtypes

camis                             int64
dba                              object
boro                             object
building                         object
street                           object
zipcode                         float64
phone                            object
cuisine_description              object
inspection_date          datetime64[ns]
action                           object
violation_code                   object
violation_description            object
critical_flag                    object
score                           float64
grade                            object
grade_date               datetime64[ns]
record_date              datetime64[ns]
inspection_type                  object
latitude                        float64
longitude                       float64
community_board                 float64
council_district                float64
census_tract                    float64
bin                             float64
bbl                             float64


## 5. Cleaning String Columns

The string columns were cleaned by removing leading and trailing whitespaces and then changing the uppercased `dba` columns to title case.

In [11]:
# Cleaning the string columns
str_cols = df.dtypes[df.dtypes == "object"].index

for col in str_cols:
    df[col] = df[col].str.strip()

    if col == "dba":
        df[col] = df[col].str.title()

## 6. Standardising the `inspection_type` column

The `inspection_type` column was divided into 'inspection_program / inspection_phase'. Text before the / was extracted to the `inspection_program` and those after extracted to `inspection_phase` using regex expresions.


In [12]:
# Check the inspection type
set(df["inspection_type"])

{'Administrative Miscellaneous / Compliance Inspection',
 'Administrative Miscellaneous / Initial Inspection',
 'Administrative Miscellaneous / Re-inspection',
 'Administrative Miscellaneous / Reopening Inspection',
 'Administrative Miscellaneous / Second Compliance Inspection',
 'Calorie Posting / Compliance Inspection',
 'Calorie Posting / Initial Inspection',
 'Calorie Posting / Re-inspection',
 'Cycle Inspection / Compliance Inspection',
 'Cycle Inspection / Initial Inspection',
 'Cycle Inspection / Re-inspection',
 'Cycle Inspection / Reopening Inspection',
 'Cycle Inspection / Second Compliance Inspection',
 'Inter-Agency Task Force / Initial Inspection',
 'Inter-Agency Task Force / Re-inspection',
 'Pre-permit (Non-operational) / Compliance Inspection',
 'Pre-permit (Non-operational) / Initial Inspection',
 'Pre-permit (Non-operational) / Re-inspection',
 'Pre-permit (Non-operational) / Second Compliance Inspection',
 'Pre-permit (Operational) / Compliance Inspection',
 'Pre-per

In [13]:
# Standardising the inspection type. It will be divided into inspection_program and inspection_phase
df["inspection_program"] =  (df["inspection_type"]
                             .str.extract(r"^([A-Za-z\s-]+).*/")[0]
                             .str.strip()
                            )

df["inspection_phase"] = (df["inspection_type"]
                          .str.extract(r".*/([A-Za-z\s-]+)")[0]
                          .str.strip()
                         )
df.head(3)

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,...,longitude,community_board,council_district,census_tract,bin,bbl,nta,location_point1,inspection_program,inspection_phase
0,50104876,Noodle Super No I,Manhattan,265,1 AVENUE,10003.0,2125290539,Chinese,2022-08-24,Violations were cited in the following area(s).,...,-73.981768,106.0,2.0,4800.0,1020423.0,1.009220e+09,MN21,NaN,Pre-permit,Second Compliance Inspection
1,50174387,Treats Of Korea,Queens,3150,STEINWAY ST,11103.0,8453232965,NaN,NaT,NaN,...,-73.918066,401.0,22.0,15500.0,4010534.0,4.006580e+09,QN70,NaN,NaN,NaN
2,50122756,Meller'S Sports Hub & Grill,Manhattan,1702,2 AVENUE,10128.0,9175964244,American,2025-08-04,Violations were cited in the following area(s).,...,-73.950670,108.0,5.0,14602.0,1050066.0,1.015510e+09,MN32,NaN,Cycle Inspection,Initial Inspection


## 7. Standardising the `cuisine_description` column

Cuisine description was standardised using two criteria;
1. Geographical origins where cuisines were categorised based on their orign such as European, African, American etc.

2. Thematic categorisation where the cuisines were caegorised based on their themes such as ethnic, fast food, beverage, etc.


In [14]:
# Check the number of distinct cuisines
df["cuisine_description"].nunique()

89

In [15]:
# inspecting the cuisine_descriptions
set(df["cuisine_description"])

{'Afghan',
 'African',
 'American',
 'Armenian',
 'Asian/Asian Fusion',
 'Australian',
 'Bagels/Pretzels',
 'Bakery Products/Desserts',
 'Bangladeshi',
 'Barbecue',
 'Basque',
 'Bottled Beverages',
 'Brazilian',
 'Cajun',
 'Californian',
 'Caribbean',
 'Chicken',
 'Chilean',
 'Chimichurri',
 'Chinese',
 'Chinese/Cuban',
 'Chinese/Japanese',
 'Coffee/Tea',
 'Continental',
 'Creole',
 'Creole/Cajun',
 'Czech',
 'Donuts',
 'Eastern European',
 'Egyptian',
 'English',
 'Ethiopian',
 'Filipino',
 'French',
 'Frozen Desserts',
 'Fruits/Vegetables',
 'Fusion',
 'German',
 'Greek',
 'Hamburgers',
 'Haute Cuisine',
 'Hawaiian',
 'Hotdogs',
 'Hotdogs/Pretzels',
 'Indian',
 'Indonesian',
 'Iranian',
 'Irish',
 'Italian',
 'Japanese',
 'Jewish/Kosher',
 'Juice, Smoothies, Fruit Salads',
 'Korean',
 'Latin American',
 'Lebanese',
 'Mediterranean',
 'Mexican',
 'Middle Eastern',
 'Moroccan',
 'New American',
 'New French',
 'Not Listed/Not Applicable',
 'Nuts/Confectionary',
 'Other',
 'Pakistani',


In [16]:
# Classifying and standardising the cuisine descriptions by geographical origins
def cuisine_standardisation(x):
    if x in ["African", "Ethiopian", "Egyptian", "Moroccan"]:
        return "African"
        
    elif x in ["Armenian", "Basque", "Czech", "Eastern European", "English", "French", "German", "Greek", "Irish", "Italian",
               "Mediterranean", "New French", "Polish", "Portuguese", "Russian", "Scandinavian", "Spanish", "Continental", "Haute Cuisine",
               "Turkish", "Pizza", "Tapas", "Australian"]:
        return "European"
        
    elif x in ["Caribbean", "Latin American", "Mexican", "Peruvian", "Brazilian", "Chilean", "Tex-Mex", "Chimichurri"]:
        return "Latin/Carribean"
        
    elif x in ["Afghan", "Asian/Asian Fusion", "Bangladeshi", "Chinese", "Chinese/Cuban", "Chinese/Japanese", "Filipino", "Fusion",
               "Indian", "Indonesian", "Iranian", "Japanese", "Korean", "Pakistani", "Southeast Asian", "Thai"]:
        return "Asian"
        
    elif x in ["American", "New American", "Soul Food", "Steakhouse", "Barbecue", "Cajun", "Creole", "Creole/Cajun", "Californian", 
               "Hamburgers", "Hotdogs", "Hotdogs/Pretzels", "Chicken", "Southwestern", "Bagels/Pretzels", "Hawaiian"]:
        return "American"
        
    elif x in ["Lebanese", "Middle Eastern", "Jewish/Kosher"]:
        return "Middle East"


    elif x in ["Bakery Products/Desserts", "Donuts", "Frozen Desserts", "Fruits/Vegetables", "Salads", "Sandwiches",
               "Sandwiches/Salads/Mixed Buffet", "Seafood", "Soups", "Soups/Salads/Sandwiches", "Pancakes/Waffles",
               "Nuts/Confectionary", "Bottled Beverages", "Beverages", "Coffee/Tea", "Juice", "Smoothies", 
               "Fruit Salads", "Juice, Smoothies, Fruit Salads", "Vegan", "Vegetarian", "Other", "Neutral"]:
        return "Neutral"

        
    elif x in ["Not Listed/Not Applicable"] or pd.isna(x):
        return "Not listed"
    
    else:
        return x

df["cuisine_geographic_category"] = df["cuisine_description"].apply(cuisine_standardisation)


In [17]:
# Classifying and standardising the cuisine descriptions based on themes

def thematic_cuisine_categorisation(x):

    if x in ["Hamburgers", "Hotdogs", "Pizza", "Sandwiches", "Donuts",
    "Chicken", "Hotdogs/Pretzels", "Bagels/Pretzels",
    "Pancakes/Waffles", "Sandwiches/Salads/Mixed Buffet", "Soups/Salads/Sandwiches"]:
        return "Quick service"

    elif x in ["Afghan", "African", "American", "Armenian", "Asian/Asian Fusion", "Australian", "Bangladeshi", "Barbecue", "Basque",
                "Brazilian", "Cajun", "Californian", "Caribbean", "Chilean", "Chimichurri", "Chinese", "Chinese/Cuban", "Chinese/Japanese",
                "Continental", "Creole", "Creole/Cajun", "Czech", "Eastern European", "Egyptian", "English", "Ethiopian",
                "Filipino", "German", "Greek", "Hawaiian", "Indian", "Indonesian", "Iranian", "Irish", "Italian", "Japanese",
                "Jewish/Kosher", "Korean", "Latin American", "Lebanese", "Mediterranean", "Mexican", "Middle Eastern",
                "Moroccan", "New American", "New French", "Pakistani", "Peruvian", "Polish", "Portuguese", "Russian",
                "Scandinavian", "Southeast Asian", "Southwestern", "Spanish", "Soul Food", "Tapas", "Tex-Mex", "Thai", "Turkish"]:
        return "Ethnic cuisine"

    elif x in ["Vegan", "Vegetarian", "Salads", "Juice, Smoothies, Fruit Salads", "Fruits/Vegetables"]:
        return "Health/Plant-based"

    elif x in ["Bakery Products/Desserts", "Frozen Desserts", "Nuts/Confectionary"]:
        return "Desserts/Bakery"


    elif x in ["Coffee/Tea", "Bottled Beverages"]:
        return "Beverage-centric"

    elif x in ["Haute Cuisine", "French", "Steakhouse"]:
        return "Fine dining"

    elif x in ["Fusion", "Seafood", "Soups", "Other"]:
        return "Others"
    else:
        return "Not listed"
        
df["thematic_cuisine_category"] = df["cuisine_description"].apply(thematic_cuisine_categorisation)


In [18]:
# Check the geographic categories
df["cuisine_geographic_category"].unique()

array(['Asian', 'Not listed', 'American', 'European', 'Neutral',
       'Latin/Carribean', 'Middle East', 'African'], dtype=object)

In [19]:
# Check the thematic categories
df["thematic_cuisine_category"].unique()

array(['Ethnic cuisine', 'Not listed', 'Desserts/Bakery', 'Quick service',
       'Beverage-centric', 'Others', 'Health/Plant-based', 'Fine dining'],
      dtype=object)

## 8. Standardising the `violation_code` column

I referred to the [Food Service Establishment Sanitary Inspection Procedures and Letter Grading](https://codelibrary.amlegal.com/codes/newyorkcity/latest/NYCrules/0-0-0-43593) to categorise the violation columns under their broad umbrellas to make analysis easier. There were 148 distinct violation codes. However, many fell under the same broad categories.


In [20]:
# Count the number of unique violation codes
df["violation_code"].nunique()

148

In [21]:
# Violation_code categorisations

# Mapping of violation code prefixes to categories
VIOLATION_CATEGORIES = {
    "02": "Time and Temperature Control for Safety",
    "03": "Food Source",
    "04": "Food Protection & Pest Control",
    "05": "Facility Design and Construction",
    "06": "Food Worker Hygiene and Other Food Protection",
    "08": "Garbage, Waste Disposal and Pest Management",
    "09": "Food Protection",
    "10": "Facility Maintenance",
    "15": "Smoking and Tobacco Use",
    "16": "Caloric & Nutritional Information",
    "18": "Administrative & Posting Violations",
    "19": "Organic & Inorganic Measures",
    "20": "Allergy & Health Measures"
}

def classify_violation(code):
    """
    Categorizes NYC restaurant health inspection violation codes into broad categories
    based on Chapter 23 of the Food Service Establishment Sanitary Inspection Procedures.

    Parameters:
        code (str): Violation code from inspection data.

    Returns:
        str: Broad category label for the violation.
    """
    if pd.isna(code):
        return "No Violation / Not Applicable"
        
    # Match based on prefix
    for prefix, category in VIOLATION_CATEGORIES.items():
        if code.startswith(prefix):
            return category
    
    return "Others / Miscellaneous"

# Apply to dataframe
df["violation_category"] = df["violation_code"].apply(classify_violation)


In [22]:
# Check the unique violations
df["violation_category"].unique()

array(['Time and Temperature Control for Safety',
       'No Violation / Not Applicable', 'Food Protection & Pest Control',
       'Facility Maintenance', 'Food Protection',
       'Food Worker Hygiene and Other Food Protection',
       'Garbage, Waste Disposal and Pest Management',
       'Others / Miscellaneous', 'Allergy & Health Measures',
       'Administrative & Posting Violations', 'Food Source',
       'Smoking and Tobacco Use', 'Organic & Inorganic Measures',
       'Facility Design and Construction',
       'Caloric & Nutritional Information'], dtype=object)

## 9. Removing irrelevant columns

Columns that were not considered useful for the analysis were dropped. They are however present in the oginal dataset.

In [23]:
# Remove irrelevant columns

df = (df
     .dropna(how = "all", axis = 1)
     .drop(columns = ["inspection_type",
                      "cuisine_description",
                      "action", 
                      "violation_description", 
                      "building", 
                      "latitude",
                      "longitude",
                      "street", 
                      "phone", 
                      "community_board", 
                      "council_district", 
                      "bin", 
                      "bbl", 
                      "nta", 
                      "census_tract"])
    )

df.head(3)

,camis,dba,boro,zipcode,inspection_date,violation_code,critical_flag,score,grade,grade_date,record_date,inspection_program,inspection_phase,cuisine_geographic_category,thematic_cuisine_category,violation_category
0,50104876,Noodle Super No I,Manhattan,10003.0,2022-08-24,02B,Critical,26.0,NaN,NaT,2025-09-18,Pre-permit,Second Compliance Inspection,Asian,Ethnic cuisine,Time and Temperature Control for Safety
1,50174387,Treats Of Korea,Queens,11103.0,NaT,NaN,Not Applicable,NaN,NaN,NaT,2025-09-18,NaN,NaN,Not listed,Not listed,No Violation / Not Applicable
2,50122756,Meller'S Sports Hub & Grill,Manhattan,10128.0,2025-08-04,04L,Critical,80.0,NaN,NaT,2025-09-18,Cycle Inspection,Initial Inspection,American,Ethnic cuisine,Food Protection & Pest Control


## 10. Handling missing values

Missing values were checked and some selected columns with mising values were imputed with appropriate data

In [24]:
# Check for misising values
df.isna().mean()

camis                          0.000000
dba                            0.000021
boro                           0.000000
zipcode                        0.010057
inspection_date                0.012805
violation_code                 0.020217
critical_flag                  0.000000
score                          0.055101
grade                          0.512437
grade_date                     0.539515
record_date                    0.000000
inspection_program             0.012805
inspection_phase               0.012805
cuisine_geographic_category    0.000000
thematic_cuisine_category      0.000000
violation_category             0.000000
dtype: float64

In [25]:
# Check for the unique values in boro
df["boro"].unique()

array(['Manhattan', 'Queens', 'Brooklyn', 'Bronx', 'Staten Island', '0'],
      dtype=object)

In [26]:
# Replace 0 in boro with "Not specified"
df["boro"] = df["boro"].replace({"0" : "Not specified"})


In [27]:
# Fill the missing grade, inspection_phase and inspection_program
df["grade"] = df["grade"].fillna("Not graded")
df["inspection_phase"] = df["inspection_phase"].fillna("Not specified")
df["inspection_program"] = df["inspection_program"].fillna("Not specified")

# confirm the imputations
df.isna().mean()

camis                          0.000000
dba                            0.000021
boro                           0.000000
zipcode                        0.010057
inspection_date                0.012805
violation_code                 0.020217
critical_flag                  0.000000
score                          0.055101
grade                          0.000000
grade_date                     0.539515
record_date                    0.000000
inspection_program             0.000000
inspection_phase               0.000000
cuisine_geographic_category    0.000000
thematic_cuisine_category      0.000000
violation_category             0.000000
dtype: float64

## 11. Export the data to mysql for further analysis

In [28]:
# Preparing the grounds for export
username = "root"
password = "MK_Of_94"
host = "localhost"
port = "3306"
database = "restaurant_health_inspection"

# Create connection engine
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")


In [29]:
# Export to sql
df.to_sql(
    name="cleaned_inspection_data",   # Table name
    con=engine,
    if_exists="replace",              # or "append" to add without deleting existing
    index=False,                      # Avoid exporting pandas index
    chunksize=1000                    # Handle large datasets in batches
)


288867

In [30]:
# Export clean data to csv for later use
df.to_csv("cleaned_inspection_data.csv", index = False)